In [1]:
import os
os.chdir('../')

In [2]:
%matplotlib inline
#%matplotlib notebook

%load_ext autoreload
%autoreload 2

In [3]:
from copy import deepcopy
from typing import List, Tuple

from cycler import cycler
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.decomposition import PCA
import scipy.stats as stats
import torch
from torch import nn
import torch.nn.functional as F
import seml

import tqdm
tqdm.tqdm.pandas()
#plt.style.use('ggplot')

In [90]:
df_experiments_1 = seml.get_results('rgnn_victims_cora',
                                  to_data_frame=True,
                                  fields=['batch_id', 'slurm', 'config', 'result'])

In [91]:
df_experiments_2 = seml.get_results('rgnn_victims_citeseer',
                                  to_data_frame=True,
                                  fields=['batch_id', 'slurm', 'config', 'result'])

In [99]:
df_experiments_3 = seml.get_results('rgnn_rpprgo_papers100M',
                                  to_data_frame=True,
                                  fields=['batch_id', 'slurm', 'config', 'result'])

In [100]:
df_experiments = df_experiments_1.append(df_experiments_2, ignore_index=True)
df_experiments = df_experiments.append(df_experiments_3, ignore_index=True)

In [101]:
df_experiments["config.model_params.label"].unique()

array(['Vanilla PPRGo', 'Soft Medoid RPPRGo (T=1.0)',
       'Soft Medoid RPPRGo (T=0.5)', 'Soft Medoid RPPRGo (T=0.2)',
       'Soft Median RPPRGo (T=1.0)', 'Soft Median RPPRGo (T=0.5)',
       'Soft Median RPPRGo (T=0.2)', 'Soft Medoid GDC (T=0.2)',
       'Vanilla GCN', 'Vanilla GDC', 'Soft Medoid GDC (T=1.0)',
       'Soft Medoid GDC (T=0.5)', 'Soft Median GDC (T=1.0)',
       'Soft Median GDC (T=0.5)', 'Soft Median GDC (T=0.2)', 'SVD GCN',
       'Jaccard GCN', 'RGCN', 'Soft Medoid PPRGo', 'Soft Median PPRGo',
       'Soft Median PPRGo (T=1.0)', 'Soft Median PPRGo (T=0.5)',
       'Soft Median PPRGo (T=0.2)', 'Soft Median PPRGo (T=5.0)',
       'Soft Median PPRGo (T=10.0)', 'Vanilla PPRGo Diffuse Embeddings'],
      dtype=object)

In [102]:
df_experiments["config.dataset"].unique()

array(['cora_ml', 'citeseer', 'ogbn-papers100M'], dtype=object)

In [103]:
model_params = {
    "Num. Layer": "nlayers",
    "Num. Filter": "n_filters",
    "Hidden Size": "hidden_size",
    "Dropout": "dropout",
    "Batch Norm": "batch_norm",
    "GDC Alpha": "gdc_params.alpha",
    "GDC K": "gdc_params.k",
    "SVD Rank": "svd_params.rank",
    "Jaccard Threshold": "jaccard_params.threshold",
    "Aggregation (Mean)": "mean",
    "Aggregation (Mean) topk": "mean_kwargs.k",
    "Aggregation (Mean) temp.": "mean_kwargs.temperature",
    "PPR Alpha": "alpha",
    "PPR Norm": "ppr_normalization",
    "PPR TopK": "topk",
    "PPR TopK eps": "eps",
}
model_params = {"config.model_params."+ v:k for k, v in model_params.items()}
model_params

{'config.model_params.nlayers': 'Num. Layer',
 'config.model_params.n_filters': 'Num. Filter',
 'config.model_params.hidden_size': 'Hidden Size',
 'config.model_params.dropout': 'Dropout',
 'config.model_params.batch_norm': 'Batch Norm',
 'config.model_params.gdc_params.alpha': 'GDC Alpha',
 'config.model_params.gdc_params.k': 'GDC K',
 'config.model_params.svd_params.rank': 'SVD Rank',
 'config.model_params.jaccard_params.threshold': 'Jaccard Threshold',
 'config.model_params.mean': 'Aggregation (Mean)',
 'config.model_params.mean_kwargs.k': 'Aggregation (Mean) topk',
 'config.model_params.mean_kwargs.temperature': 'Aggregation (Mean) temp.',
 'config.model_params.alpha': 'PPR Alpha',
 'config.model_params.ppr_normalization': 'PPR Norm',
 'config.model_params.topk': 'PPR TopK',
 'config.model_params.eps': 'PPR TopK eps'}

In [110]:
train_params = {
    "Learning Rate": "lr",
    "Weight Decay": "weight_decay",
    "Max Epochs": "max_epochs",
    "Patience": "patience",
    "Batch Size": "batch_size",
    "Annealing Scheduler": "use_annealing_scheduler",
    #"AS with Warm Restart": "scheduler_warm_restarts",
}
train_params = {"config.train_params."+ v:k for k, v in train_params.items()}
train_params

{'config.train_params.lr': 'Learning Rate',
 'config.train_params.weight_decay': 'Weight Decay',
 'config.train_params.max_epochs': 'Max Epochs',
 'config.train_params.patience': 'Patience',
 'config.train_params.batch_size': 'Batch Size',
 'config.train_params.use_annealing_scheduler': 'Annealing Scheduler'}

In [111]:
labels_to_plot = [
    'Vanilla GCN',
    'Vanilla GDC',
    'SVD GCN',
    'Jaccard GCN',
    'RGCN',
    'Soft Medoid GDC (T=0.5)',
    #'Soft Median GDC (T=1.0)',
    #'Soft Median GDC (T=0.5)',
    'Soft Median GDC (T=0.2)'
]

In [112]:
dataset_map = {
    'cora_ml': r'\textbf{Cora ML}', # '\rotatebox{90}{Cora ML~\citep{Bojchevski2018}}',
    'citeseer': r'\textbf{Citeseer}', # ~\citep{McCallum2000}
    'pubmed': r'\textbf{PubMed}', # ~\citep{Sen2008}
    'ogbn-arxiv': r'\textbf{arXiv}', # ~\citep{Hu2020}
    'ogbn-products': r'\textbf{Products}', # ~\citep{Hu2020},
    'ogbn-papers100M': r'\textbf{Papers 100M}'
}
dataset_order = [dataset_map[k] for k in dataset_map.keys()]

In [116]:
dataset_order

['\\textbf{Cora ML}',
 '\\textbf{Citeseer}',
 '\\textbf{PubMed}',
 '\\textbf{arXiv}',
 '\\textbf{Products}',
 '\\textbf{Papers 100M}']

In [126]:
groups_experiment = df_experiments.groupby(["config.dataset", "config.model_params.label"])

In [129]:
df_hpt = pd.DataFrame(columns=["label", "dataset", "seeds"] + list(train_params.keys()) + list(model_params.keys()))
for (dataset, label), max_idx in groups_experiment.idxmax()["result.accuracy"].iteritems():
    df_hpt.loc[len(df_hpt)] = [label, dataset, []] + list(df_experiments.iloc[max_idx][list(train_params.keys()) + list(model_params.keys())])

In [130]:
df_hpt#.fillna("-")

,label,dataset,seeds,config.train_params.lr,config.train_params.weight_decay,config.train_params.max_epochs,config.train_params.patience,config.train_params.batch_size,config.train_params.use_annealing_scheduler,config.model_params.nlayers,config.model_params.n_filters,config.model_params.hidden_size,config.model_params.dropout,config.model_params.batch_norm,config.model_params.gdc_params.alpha,config.model_params.gdc_params.k,config.model_params.svd_params.rank,config.model_params.jaccard_params.threshold,config.model_params.mean,config.model_params.mean_kwargs.k,config.model_params.mean_kwargs.temperature,config.model_params.alpha,config.model_params.ppr_normalization,config.model_params.topk,config.model_params.eps
0,Jaccard GCN,citeseer,[],0.010,0.0005,300,200,NaN,NaN,NaN,64,NaN,0.5,NaN,NaN,NaN,NaN,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RGCN,citeseer,[],0.010,0.0005,300,200,NaN,NaN,NaN,64,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SVD GCN,citeseer,[],0.010,0.0005,300,200,NaN,NaN,NaN,64,NaN,0.5,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Soft Median GDC (T=0.2),citeseer,[],0.010,0.0005,300,200,NaN,NaN,NaN,64,NaN,0.5,NaN,0.15,64.0,NaN,NaN,soft_median,NaN,0.2,NaN,NaN,NaN,NaN
4,Soft Median GDC (T=0.5),citeseer,[],0.010,0.0005,300,200,NaN,NaN,NaN,64,NaN,0.5,NaN,0.15,64.0,NaN,NaN,soft_median,NaN,0.5,NaN,NaN,NaN,NaN
5,Soft Median GDC (T=1.0),citeseer,[],0.010,0.0005,300,200,NaN,NaN,NaN,64,NaN,0.5,NaN,0.15,64.0,NaN,NaN,soft_median,NaN,1.0,NaN,NaN,NaN,NaN
6,Soft Median RPPRGo (T=0.2),citeseer,[],0.001,0.0100,300,200,NaN,NaN,2.0,64,64.0,0.5,1.0,NaN,NaN,NaN,NaN,soft_median,NaN,0.2,0.116547,row,64.0,0.0010
7,Soft Median RPPRGo (T=0.5),citeseer,[],0.010,0.0100,300,200,NaN,NaN,3.0,64,64.0,0.5,1.0,NaN,NaN,NaN,NaN,soft_median,NaN,0.5,0.104085,row,64.0,0.0010
8,Soft Median RPPRGo (T=1.0),citeseer,[],0.010,0.0010,300,200,NaN,NaN,3.0,64,128.0,0.5,1.0,NaN,NaN,NaN,NaN,soft_median,NaN,1.0,0.219912,row,64.0,0.0010
9,Soft Medoid GDC (T=0.2),citeseer,[],0.010,0.0005,300,200,NaN,NaN,NaN,64,NaN,0.5,NaN,0.15,64.0,NaN,NaN,soft_k_medoid,64.0,0.2,NaN,NaN,NaN,NaN
